In [17]:
import numpy as np
import pandas as pd
import os
import pickle 
import procyclingstats
import sys
sys.path.insert(1, '/mnt/HDD/Code/TDF/src')
import power_helper as ph
import plotly.express as px

In [18]:
df = pd.read_csv("../Data/Data_edited.csv")
# Riders strikes
df = df.drop(axis=0, index=248) # 1996 stage 9
df = df.drop(axis=0, index=234) # 1995 stage 16
# Calculate power for full tour with fixed grade
total_climbed = df.groupby("year").stage_vertical_meters.sum()
total_distance = df.groupby("year").stage_distance.sum()
gc_weights = df.groupby("year").gc_weight.mean()
gc_time = df.groupby("year").gc_stage_time.sum()
total_grades = total_climbed/(total_distance*1000)
velocity = (total_distance*1000)/gc_time
year_power = ph.cycling_power(total_grades, gc_weights, velocity)

In [5]:
['year', 'Departure', 'Arrival', 'Type', 'Profile', 'Stage winner', 'GC leader', 'Time',  'Distance', 'Vertical meters', 'Speed', 'Power'][2]

'Arrival'

In [28]:
a = df.iloc[2].reset_index()['index'].values

In [29]:
a

array(['Unnamed: 0', 'year', 'Race_url', 'stage', 'stage_url',
       'stage_type', 'stage_winner_time_str', 'profile_icon',
       'stage_winner', 'stage_winner_url', 'stage_winner_time',
       'stage_distance', 'gc_stage_time_str', 'stage_vertical_meters',
       'gc_leader', 'gc_leader_url', 'gc_stage_time', 'power',
       'gc_weight', 'gc_height', 'stage_grade', 'gc_speed',
       'stage_departure', 'stage_arrival'], dtype=object)

In [ ]:
# Power to speed

# avg_grades =  df.groupby("year").stage_vertical_meters.sum() / (df.groupby("year").stage_distance.sum()*1000) 
avg_speed = ((df.groupby("year").stage_distance.sum()*1000) / df.groupby("year").gc_stage_time.sum()) * 3.6

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df.year.unique(), y=df.groupby("year").power.mean(), name="power"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.year.unique(), y=avg_speed, name="speed"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df.year.unique(), y=year_power, name="year_power")
)

# Add figure title
fig.update_layout(
    title_text="Power and speed by year"
)

# Set x-axis title
fig.update_xaxes(title_text="Year")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Power</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Grade</b>", secondary_y=True)
fig.update_yaxes(range = [0,500])
fig.update_yaxes(range = [30,50], secondary_y=True)

fig.show()

In [ ]:
px.scatter(color=df.year.unique(), x=df.groupby("year").stage_vertical_meters.sum(), y=df.groupby("year").stage_distance.sum(), title="Distance compared to elevation gain over the years")


In [ ]:
px.scatter(df,x="year", y="stage_grade", title="Stage grades per year")


In [ ]:

fig = px.scatter(df.groupby("year").power.median(), title="Median stage power compared to full tour power")
fig.add_scatter(x=year_power.index, y=year_power.values)
fig.show()

In [ ]:
px.scatter(df[(df.stage_distance > 50) & (df.stage_type != "ITT")], x="year", y="power", hover_data="stage_url", color="stage_distance", trendline="ols")


In [ ]:
px.box(df[(df.stage_distance > 50) & (df.stage_type != "ITT")], x="year", y="power", hover_data="stage_url", color="year")